In [12]:
import sys
from datasets import load_dataset
import torch
from trl.trainer.dpo_trainer import DPOTrainer

sys.path.append('..')
from model.causal_lm import AutoLMFineTuner
from model.utils import LMHyperParams, SmModel, ModelChoice
from train import start_training
%load_ext autoreload
%autoreload 2

2024-11-14 00:29:41.447 | INFO     | train:<module>:5 - Loading dependencies - Torch...


2024-11-14 00:29:41.557 | INFO     | train:<module>:16 - Loading dependencies - Lightning...
2024-11-14 00:29:41.558 | INFO     | train:<module>:25 - Loading dependencies - Project...


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from dataset.squad import DollyEntityExtractionDataModule

dataset = load_dataset("roborovski/codecontests-dpo")

In [10]:
params = LMHyperParams(base_model_checkpoint="Qwen/Qwen2.5-0.5B")
model = AutoLMFineTuner(params)
data_module = DollyEntityExtractionDataModule(16, model.tokenizer, 1024)

In [13]:
start_training(
    "16-mixed", "", model, model.tokenizer, data_module, params, False, ""
)


Seed set to 42


2024-11-14 00:29:45.847 | INFO     | train:start_training:268 - Effective batch size: 8 (2x4)
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /admin/home-brianf/smolmodels/.venv/lib/python3.11/s ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:44: Attribute 'params' removed from hparams because it cannot be pickled. You can suppress this warning by setting `self.save_hyperparameters(ignore=['params'])`.
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.s

Configuring optimizers: 0


/admin/home-brianf/smolmodels/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=175` in the `DataLoader` to improve performance.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ValueError: Either warmup_steps_count or warmup_ratio must be set